In [7]:
import os
import pandas as pd
from pathlib import Path

In [8]:
PATH = Path.cwd()

In [9]:
EC35_DIR = Path.cwd().parent / "multimodal_er" / "EmoComics35"
XLS_DIR = EC35_DIR / "raw_files" / "xls"

In [10]:
### Read

In [11]:
xlsx_files = [f for f in os.listdir(XLS_DIR) if f.endswith('.xlsx')]

In [12]:
len(xlsx_files)

35

In [18]:
dfs = []
for file in xlsx_files:
    file_path = os.path.join(XLS_DIR, file)
    df = pd.read_excel(file_path)
    df["source_file"] = file
    dfs.append(df)

In [19]:
ec35_df = pd.concat(dfs, ignore_index=True)

In [20]:
ec35_df = ec35_df.dropna(subset=["Annotations"])